In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_config import options, USERNAME, PASSWORD, PATTERN_BROAD_TEXT, PATTERN_LINK_TEXT
from bs4 import BeautifulSoup
import time
import re
#from scraper_functions import collect_data, scroll_to_bottom, driver_search, scroll_once, collect_page, collector, collect_elem_by_elem
import pandas as pd

In [2]:
from selenium_config import PATTERN_BROAD_TEXT, PATTERN_LINK_TEXT, PATTERN_HEADLINE_TEXT
import re
import time
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

def collect_data(source_page):
    title = re.findall(PATTERN_HEADLINE_TEXT, source_page)

    collected = []
    for match in re.finditer(PATTERN_BROAD_TEXT, source_page):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    if not collected:
        return ["NULL"], "NULL"

    # Find span for broad text
    first_broad_span = collected[0][0][0]
    last_broad_span = collected[-1][0][1]

    # correct span for broad texts now when broadtext chunk span is found
    collected = [((n[0][0] - first_broad_span, n[0][1] - first_broad_span), n[1]) for n in collected]

    for match in re.finditer(PATTERN_LINK_TEXT, source_page[first_broad_span:last_broad_span]):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    collected.sort(key=lambda s: s[0])
    text_final = ""
    for data in collected:
        text_final += data[1]

    if not title:
        title = ["NULL"]

    return title[0], text_final


def driver_search(driver, keywords="mord, göteborg", search_box_class_name="css-9rv3gz"):
    time.sleep(1)
    search = driver.find_element_by_class_name(search_box_class_name)
    time.sleep(1)
    search.send_keys(keywords)
    search.submit()
    time.sleep(1)


def collect_elem_by_elem(driver, element_class):
    try:
        titles = []
        texts=[]
        print("\n collecting...")
        # element = driver.find_element_by_class_name(element_class)
        elems = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".css-3ks4jq [href]")))
        links = [elem.get_attribute('href') for elem in elems]
        print(links)
        last_elem = elems[-1]
        print("location of last element", last_elem.location)
        # for e in range(len(elems)):

        # get current main window handle
        cur_win = driver.current_window_handle

        for element in elems:
            # WebDriverWait(driver, 10).until(EC.element_to_be_clickable(e))
            # if elems[e].is_displayed():
            #     print("IT IS DISPLAYED", e)

            driver.switch_to_window([win for win in driver.window_handles if win != cur_win[0]])
            element.click()
            # time.sleep(5)
            # print("\n SOURCE -->", driver.page_source)
            title, text = collect_data(driver.page_source)
            titles.append(title)
            texts.append(text)
            df = pd.DataFrame()
            df["titles"] = titles
            df["texts"] = texts
            df.to_csv(f"./Collected/data.csv")

            driver.close()  # close new window
            driver.switch_to_window(cur_win)  # switch back to main window


        print("\n Scrolling...")
        scroll_to_element(driver, last_elem)




        print("\n scrolling...")
    except RuntimeError:
        print("Oops!  Something went wrong.  Try again...")


def scroll_to_element(driver, element):
    # this scrolls until the element is in the middle of the page
    desired_y = (element.size['height'] / 2) + element.location['y']
    current_y = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script(
        'return window.pageYOffset')
    scroll_y_by = desired_y - current_y
    driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)


In [3]:
DRIVER_PATH = "./Chrome-driver/chromedriver.exe"
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get("https://www.aftonbladet.se/sok")

# Do the search
driver_search(driver, keywords="mord, göteborg", search_box_class_name="css-9rv3gz")
print("\n searching...")


 searching...


In [4]:
titles = []
texts=[]
print("\n collecting...")
# element = driver.find_element_by_class_name(element_class)
elems = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".css-3ks4jq [href]")))
links = [elem.get_attribute('href') for elem in elems]
print(links)
last_elem = elems[-1]
print("location of last element", last_elem.location)
# for e in range(len(elems)):




 collecting...
['https://www.aftonbladet.se/nyheter/a/GGkbjq/anhallen-efter-misstankt-mord-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/Qm5XXx/misstankt-mord-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/Joyb46/misstankt-mord-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/Al20Jx/man-gripen-for-mord-vid-heden-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/rAkno3/atalas-for-mord-pa-kvinna-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/0K9xkB/tva-haktade-for-mord-i-goteborg--en-anhallen', 'https://www.aftonbladet.se/nyheter/a/6zkwg3/man-atalad-for-mord-vid-heden-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/P9Wq3b/misstankt-mord-i-centrala-goteborg', 'https://www.aftonbladet.se/nyheter/a/6jpKL8/dod-man-i-centrala-goteborg-utreds-som-mord', 'https://www.aftonbladet.se/nyheter/a/naOrX5/misstankt-mord-i-goteborg', 'https://www.aftonbladet.se/nyheter/a/8mVJmx/man-hittad-dod-i-bostad-i-goteborg--misstankt-mord', 'https://www.aftonbladet.se/nyheter/a/2GemPR/tva-hakt

In [5]:
# get current main window handle
cur_win = driver.current_window_handle

In [6]:
cur_win

'CDwindow-EA112F23949BB3A4C891BED3D2CB651A'

In [7]:
driver.window_handles

['CDwindow-EA112F23949BB3A4C891BED3D2CB651A']

In [ ]:
for element in elems:
    # WebDriverWait(driver, 10).until(EC.element_to_be_clickable(e))
    # if elems[e].is_displayed():
    #     print("IT IS DISPLAYED", e)

    driver.switch_to_window([win for win in driver.window_handles if win != cur_win[0]])
    element.click()
    # time.sleep(5)
    # print("\n SOURCE -->", driver.page_source)
    title, text = collect_data(driver.page_source)
    titles.append(title)
    texts.append(text)
    df = pd.DataFrame()
    df["titles"] = titles
    df["texts"] = texts
    df.to_csv(f"./Collected/data.csv")

    driver.close()  # close new window
    driver.switch_to_window(cur_win)  # switch back to main window